# Números Primos e Algoritmos de Geração
## 1. Introdução

Números primos são inteiros maiores que 1 que só podem ser divididos por 1 e por eles mesmos.

**Importância:**
- Criptografia moderna (RSA, Diffie-Hellman, etc.)
- Teoria dos números
- Otimização e algoritmos

**Objetivo deste notebook:**
Implementar e comparar diferentes algoritmos de geração de números primos, medindo tempo e uso de memória.

In [2]:
import math

def eh_primo(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    limite = int(math.sqrt(n)) + 1
    for i in range(3, limite, 2):
        if n % i == 0:
            return False
    return True

print(eh_primo(29))

True



# Teste de Mesa – `eh_primo(29)`

**Entrada:** `n = 29`  

1. Verifica `n < 2` → ❌ (29 não é menor que 2)  
2. Verifica `n == 2` → ❌ (29 ≠ 2)  
3. Verifica `n % 2 == 0` → ❌ (29 não é par)  
4. Calcula limite: $\lfloor \sqrt{29} \rfloor + 1 = 6$  
5. Laço `for i in range(3, 6, 2)` → valores de `i`: **3, 5**  
   - `i = 3` → $29 \bmod 3 \neq 0$ → continua  
   - `i = 5` → $29 \bmod 5 \neq 0$ → continua  

**✅ Resultado final:** 29 é primo  


In [3]:
def crivo_eratostenes(n):
    primos = [True] * (n+1)
    primos[0] = primos[1] = False
    for i in range(2, int(math.sqrt(n)) + 1):
        if primos[i]:
            for j in range(i*i, n+1, i):
                primos[j] = False
    return [i for i, primo in enumerate(primos) if primo]

print(crivo_eratostenes(30))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]



# Teste de Mesa – `crivo_eratostenes(30)`

1. Início: `primos = [True] * 31`  
2. Ajuste: `primos[0] = primos[1] = False`  
3. Loop `i` até $\sqrt{30}$ → `i = 2, 3, 4, 5`  

- `i = 2` → elimina múltiplos (4, 6, 8, …, 30)  
- `i = 3` → elimina múltiplos (9, 12, 15, …, 30)  
- `i = 4` → já marcado como falso, pula  
- `i = 5` → elimina múltiplos (25, 30)  

**✅ Restam:** [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]  


In [4]:
import numpy as np

def crivo_numpy(n):
    primos = np.ones(n+1, dtype=bool)
    primos[0:2] = False
    for i in range(2, int(n**0.5) + 1):
        if primos[i]:
            primos[i*i:n+1:i] = False
    return np.nonzero(primos)[0]

print(crivo_numpy(30))

[ 2  3  5  7 11 13 17 19 23 29]



# Teste de Mesa – `crivo_numpy(30)`

1. Cria vetor booleano `primos = np.ones(31, dtype=bool)`  
2. Ajuste: `primos[0:2] = False`  
3. Loop `i` até $\sqrt{30}$ → elimina múltiplos de cada primo de forma vetorizada:  

- `i = 2` → `primos[4::2] = False`  
- `i = 3` → `primos[9::3] = False`  
- `i = 4` → já está falso, pula  
- `i = 5` → `primos[25::5] = False`  

**✅ Resultado final:** mesmo do Eratóstenes → [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]  


In [5]:
def crivo_segmentado(n):
    limite = int(math.sqrt(n)) + 1
    primos_base = crivo_eratostenes(limite)
    primos = primos_base.copy()
    inicio = limite
    while inicio <= n:
        fim = min(inicio + limite, n+1)
        segmento = [True] * (fim - inicio)
        for p in primos_base:
            inicio_mult = max(p*p, (inicio + p - 1) // p * p)
            for j in range(inicio_mult, fim, p):
                segmento[j - inicio] = False
        for i, val in enumerate(segmento):
            if val:
                primos.append(i + inicio)
        inicio += limite
    return primos

print(crivo_segmentado(30))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]



# Teste de Mesa – `crivo_segmentado(30)`

1. Calcula `limite = ⌊√30⌋ + 1 = 6`  
2. `primos_base = crivo_eratostenes(6)` → [2, 3, 5]  
3. `primos = [2, 3, 5]`  

**Segmentos:**  

- `[6, 12)` → elimina múltiplos de 2, 3, 5 → sobra [7, 11]  
- `[12, 18)` → elimina múltiplos de 2, 3, 5 → sobra [13, 17]  
- `[18, 24)` → elimina múltiplos de 2, 3, 5 → sobra [19, 23]  
- `[24, 30]` → elimina múltiplos de 2, 3, 5 → sobra [29]  

**✅ Resultado final:** [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]  

In [6]:
## 4. Benchmark de desempenho

import timeit                # Para medir tempo de execução de funções com precisão
import psutil, tracemalloc, pandas as pd
# psutil -> obtém informações detalhadas sobre o processo Python (CPU, memória)
# tracemalloc -> rastreia a memória alocada pelo Python, permite capturar picos
# pandas -> cria DataFrames para organizar resultados e compará-los facilmente

# Função que mede tempo de execução e uso de memória de qualquer função
def medir_tempo_memoria(func, *args):
    tracemalloc.start()  # Inicia o rastreamento da memória Python
    # tracemalloc rastreia apenas memória alocada pelo Python, não pelo sistema externo
    # Isso é útil para saber quanta memória o seu código realmente está usando internamente
    
    processo = psutil.Process()  # Representa o processo Python atual
    mem_inicial = processo.memory_info().rss  
    # RSS (Resident Set Size) -> quantidade de memória física usada pelo processo
    # Diferente do tracemalloc, isso inclui tudo que o processo consome, não só objetos Python

    inicio = timeit.default_timer()  # Marca o tempo de início da execução
    resultado = func(*args)          # Executa a função com os argumentos fornecidos
    fim = timeit.default_timer()     # Marca o tempo de término

    mem_final = processo.memory_info().rss  # Memória após execução
    pico, _ = tracemalloc.get_traced_memory()  
    # Pega o pico de memória alocada pelo Python durante a execução
    tracemalloc.stop()  
    # Para o rastreamento do Python, liberando recursos

    # Calcula memória total usada:
    # mem_final - mem_inicial -> diferença real do processo no sistema
    # pico -> memória interna do Python, para capturar picos temporários que RSS sozinho não mostra
    memoria_MB = (mem_final - mem_inicial + pico) / (1024*1024)  # Converte de bytes para MB

    # Retorna dicionário com métricas:
    return {
        "tempo (s)": fim - inicio,      # Tempo total de execução
        "memória (MB)": memoria_MB,     # Memória total usada (inclui pico do Python)
        "qtde primos": len(resultado)   # Quantidade de primos encontrados (assumindo lista de primos)
    }

n = 100000  # Limite superior para a geração de primos

resultados = []  # Lista que armazenará métricas de cada algoritmo
algoritmos = {
    "Crivo Eratóstenes": crivo_eratostenes,
    "Crivo NumPy": crivo_numpy,
    "Crivo Segmentado": crivo_segmentado
}
# Dicionário com nomes legíveis dos algoritmos e suas funções correspondentes

# Loop para medir cada algoritmo
for nome, func in algoritmos.items():
    r = medir_tempo_memoria(func, n)  # Executa benchmark
    r["algoritmo"] = nome             # Adiciona o nome do algoritmo ao resultado
    resultados.append(r)              # Armazena na lista

df = pd.DataFrame(resultados)  # Converte resultados em um DataFrame do pandas
df                             # Exibe tabela com tempo, memória e quantidade de primos

,tempo (s),memória (MB),qtde primos,algoritmo
0,0.753824,2.599342,9592,Crivo Eratóstenes
1,0.002441,0.073914,9592,Crivo NumPy
2,1.197910,0.416794,9593,Crivo Segmentado


# Detalhes importantes sobre memória

- `RSS (psutil):` memória usada pelo processo inteiro, incluindo bibliotecas, buffers e objetos Python. Pode não refletir apenas o que o seu código gerou.

- `tracemalloc:` rastreia apenas objetos Python, útil para detectar picos de uso temporário que podem passar despercebidos pelo RSS.

- `Soma de RSS + pico do tracemalloc:` uma estimativa segura do uso real de memória pelo algoritmo, capturando tanto o consumo persistente quanto os picos momentâneos.

- `Por que medir ambos:` alguns algoritmos podem usar muita memória temporária que é liberada rapidamente; tracemalloc captura isso, RSS sozinho não.

## 5. Conclusão

- O **teste simples** é didático, mas inviável para valores grandes.
- O **crivo de Eratóstenes** é eficiente e fácil de implementar.
- O **crivo com NumPy** é ainda mais rápido por conta da vetorização.
- O **crivo segmentado** é útil para gerar primos muito grandes sem estourar a memória.

---
Em resumo:  
- Para aprendizado → use o método simples e o crivo clássico.  
- Para eficiência prática → use NumPy ou segmentado.  